# This notebook presents the different pre-processing and data augmentation techniques used on the Stanford dataset


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jeremynadal33/classify_stanford_dogs/blob/master/notebooks/pre-processing_data-augmentation.ipynb)


There are the different data augmentation steps:
* Changing to gray scale
* Equalizing the images
* Vertical and horizontal flipping
* Various rotations
* Random zooms (and the check to know wether or not the dog is still on the image
* Random translations (same)


In [ ]:
import cv2
import os 
import cv2
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  
  root_dir = '/content/gdrive/My Drive/Formation-OC/P6-Images/'
  input_dir = root_dir + 'inputs/'
  png_dir = root_dir + 'pngs/'

  #my script
  !ls gdrive/MyDrive/Formation-OC/P6-Images/
else:
  print('Not running on CoLab')
  #my script
  root_dir = '/Users/jeremynadal/Documents/Formation OC IML/P6/'
  input_dir = root_dir + 'inputs/'
  png_dir = root_dir + 'pngs/'
  model_dir = root_dir +'models/'

In [ ]:
species = os.listdir(input_dir+'sep_images/train/')[:4] #Just 4 to try out
base_dir = input_dir+'sep_images/train/'

## Lets see the effect of whitening and equalization

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]).astype(int)

def CLAHE_rgb(img): 
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

  new = np.zeros(img.shape, dtype = int)
  new[:,:,0] = clahe.apply(img[:,:,0])
  new[:,:,1] = clahe.apply(img[:,:,1])
  new[:,:,2] = clahe.apply(img[:,:,2])

  return new

In [ ]:
path_test = base_dir + os.listdir(base_dir)[0]

fig, axes = plt.subplots(4, 4, figsize=(20, 15))

for specie in range(len(species)):
  path = base_dir + species[specie] +'/' + os.listdir(base_dir + species[specie])[0]
  img = cv2.imread(path,1)

  imgs = [img]
  imgs.append( tf.image.per_image_standardization(img) )
  imgs.append( equalize_rgb(img) )
  imgs.append( CLAHE_rgb(img) ) 

  for idx in range(len(imgs)):
    axes[specie,idx].imshow(imgs[idx])
    axes[specie,idx].axis('off')

axes[0,0].set_title('Original image')
axes[0,1].set_title('Standardised image')
axes[0,2].set_title('Equalized image')
axes[0,3].set_title('CLAHE image')

axes[0,0].set_ylabel(species[0].split('-')[1])
axes[0,1].set_ylabel(species[1].split('-')[1])
axes[0,2].set_ylabel(species[2].split('-')[1])
axes[0,3].set_ylabel(species[3].split('-')[1]) 


plt.savefig(png_dir+'pre-processing.png')

plt.show()


## Make a new directory to train and put specificaly augmented dataFirst, lets define each function to modify the images. 
Here, we will :
* Flip horizontally 
* Flip vertically 
* Horizontal shift
* Vertical shift
* Rotates the images
* Zoom 
* Modify brightness

Make a new directory to train and put specificaly augmented data

In [ ]:
import random
from google.colab.patches import cv2_imshow

def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.BORDER_REPLICATE)
    return img


def shift(img, horizontal = False, vertical = False, ratio=0.0):
  
  assert not (ratio<-1) or (ratio>1), 'ratio must be between -1 and 1'
  assert ((horizontal) or (vertical)) and not ((horizontal) and (vertical)), 'Either horizontal or vertical must be True'
  
  #ratio = random.uniform(-ratio, ratio)
  h, w = img.shape[:2]

  if horizontal : 
    to_shift = w*ratio
    if ratio > 0:
      img = img[:, :int(w-to_shift), :]
    if ratio < 0:
      img = img[:, int(-1*to_shift):, :]
  elif vertical : 
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
  
  img = fill(img, h, w)
  return img


def flip(img, vertical = False, horizontal = False ):
  assert ((horizontal) or (vertical)) and not ((horizontal) and (vertical)), 'Either horizontal or vertical must be True'

  if horizontal : 
    img = cv2.flip(img, 1)
  elif vertical : img = cv2.flip(img, 0)

  return img


def rotation(img, angle):
    #angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))

    img = fill(img, h, w)
    return img


def zoom(img, value):
    assert not (value<0) or (value>1), 'value must be between -1 and 1'

    if value <= 1 : 
      h, w = img.shape[:2]
      h_taken = int(value*h)
      w_taken = int(value*w)
      h_start = random.randint(0, h-h_taken)
      w_start = random.randint(0, w-w_taken)
      img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
      img = fill(img, h, w)
    if value > 1 : 
      h, w = img.shape[:2]
      new_h = int(value*h)
      new_w = int(value*w)
      h_start = new_h - int((h+new_h)/2)
      w_start = new_w - int((w+new_w)/2)

      new_img = np.zeros((new_h, new_w,3),dtype=int)

      new_img[h_start:h_start+h, w_start:w_start+w, :] = img 

      img = new_img

    return img

def brightness(img, low=1, high=3):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

def histogram_equalization(img_in):
  # segregate color streams
  b,g,r = cv2.split(img_in)
  h_b, bin_b = np.histogram(b.flatten(), 256, [0, 256])
  h_g, bin_g = np.histogram(g.flatten(), 256, [0, 256])
  h_r, bin_r = np.histogram(r.flatten(), 256, [0, 256])
  # calculate cdf    
  cdf_b = np.cumsum(h_b)  
  cdf_g = np.cumsum(h_g)
  cdf_r = np.cumsum(h_r)
    
  # mask all pixels with value=0 and replace it with mean of the pixel values 
  cdf_m_b = np.ma.masked_equal(cdf_b,0)
  cdf_m_b = (cdf_m_b - cdf_m_b.min())*255/(cdf_m_b.max()-cdf_m_b.min())
  cdf_final_b = np.ma.filled(cdf_m_b,0).astype('uint8')
  
  cdf_m_g = np.ma.masked_equal(cdf_g,0)
  cdf_m_g = (cdf_m_g - cdf_m_g.min())*255/(cdf_m_g.max()-cdf_m_g.min())
  cdf_final_g = np.ma.filled(cdf_m_g,0).astype('uint8')
  cdf_m_r = np.ma.masked_equal(cdf_r,0)
  cdf_m_r = (cdf_m_r - cdf_m_r.min())*255/(cdf_m_r.max()-cdf_m_r.min())
  cdf_final_r = np.ma.filled(cdf_m_r,0).astype('uint8')
  # merge the images in the three channels
  img_b = cdf_final_b[b]
  img_g = cdf_final_g[g]
  img_r = cdf_final_r[r]
  
  img_out = cv2.merge((img_b, img_g, img_r))
  # validation
  equ_b = cv2.equalizeHist(b)
  equ_g = cv2.equalizeHist(g)
  equ_r = cv2.equalizeHist(r)
  equ = cv2.merge((equ_b, equ_g, equ_r))
  
  return img_out


def handmade_augmentation(input_dir, output_dir):
  assert os.path.exists(input_dir), 'input_dir doesnot exist'
  assert os.path.exists(output_dir), 'output_dir doesnot exist, please create it first'
  
  if output_dir[-1]!='/': output_dir+'/'
  if input_dir[-1]!='/': input_dir+'/'

  species = os.listdir(input_dir) 

  assert os.path.isdir(input_dir+species[0]), 'input_dir must contain directories of images'

  
  print('Looking for images and returning 12 times as many images')
  for output_fold in os.listdir(output_dir):
    os.rmdir(output_dir+output_fold)

  for specie in species :
    os.mkdir(output_dir+specie)
    print('Dealing with ', specie.split('-')[1], '\nFound {} images'.format(len(os.listdir(input_dir+specie))))

    for file in os.listdir(input_dir+specie) :

      img = plt.imread(input_dir+specie+'/'+file)
      imgs = [img]
      imgs.append( shift(img, ratio = 0.25, horizontal= True) ) 
      imgs.append( shift(img, ratio = 0.25, vertical= True) )

      imgs.append( flip(img, horizontal= True) )
      imgs.append( flip(img, vertical= True) )

      imgs.append( rotation(img, 45) )
      imgs.append( rotation(img, -45) )
      imgs.append( rotation(img, 135) )

      imgs.append( zoom(img, 0.9) )

      imgs.append( zoom(img, 1.5) )

      imgs.append( brightness(img, 1, 3) )

      imgs.append( histogram_equalization(img) )

      for idx in range(len(imgs)) :
        name = output_dir+specie+'/aug_'+str(file).split('.')[0]+'_'+str(idx)+'.jpg'
        cv2.imwrite(name, imgs[idx])


In [ ]:
img_path = baseline_dir+'train/'+os.listdir(baseline_dir+'train/')[3]+'/'+ os.listdir(baseline_dir+'train/'+os.listdir(baseline_dir+'train/')[3])[0]

img = plt.imread( img_path, 0  )

h, w = img.shape[:2]

#img = fill(img, h, w)

img_shift_h = shift(img, ratio = 0.25, horizontal= True)
img_shift_v = shift(img, ratio = 0.25, vertical= True)

img_flip_h = flip(img, horizontal= True)
img_flip_v = flip(img, vertical= True)

img_rot_1 = rotation(img, 45)
img_rot_2 = rotation(img, -45)
img_rot_3 = rotation(img, 135)

img_zoom_in = zoom(img, 0.9)

img_zoom_out = zoom(img, 1.5)

img_brightness = brightness(img, 1, 3)

img_equalized = histogram_equalization(img)

In [ ]:
fig, axes = plt.subplots(2, 6, figsize=(20,10), sharex=False)

axes = axes.flatten()

axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title('Original img')

axes[1].imshow(img_shift_h)
axes[1].axis('off')
axes[1].set_title('img horizontally shifted')

axes[2].imshow(img_shift_v)
axes[2].axis('off')
axes[2].set_title('img vertically shifted')


axes[3].imshow(img_flip_v)
axes[3].axis('off')
axes[3].set_title('img vertically fliped')

axes[4].imshow(img_flip_h)
axes[4].axis('off')
axes[4].set_title('img horizontal fliped')


axes[5].imshow(img_rot_1)
axes[5].axis('off')
axes[5].set_title('img 45 degres rotated')

axes[6].imshow(img_rot_2)
axes[6].axis('off')
axes[6].set_title('img -45 degres rotated')

axes[7].imshow(img_rot_3)
axes[7].axis('off')
axes[7].set_title('img 135 degres rotated')

axes[8].imshow(img_zoom_in)
axes[8].axis('off')
axes[8].set_title('img zoomed in')

axes[9].imshow(img_zoom_out)
axes[9].axis('off')
axes[9].set_title('img zoomed out')

axes[10].imshow(img_brightness)
axes[10].axis('off')
axes[10].set_title('img increased brightness')

axes[11].imshow(img_equalized)
axes[11].axis('off')
axes[11].set_title('img equalized')

plt.savefig(root_dir+'pngs/handmade_augmentation.png')
plt.show()

In [ ]:
handmade_augmentation(input_dir= input_dir+'baseline_inputs/train/', output_dir= input_dir+'baseline_inputs/train_aug/')

## Now lets instanciate an image augmentation object from tf.keras to understand its behavior
##Lets build two imagedatagenerator : one will be used for data augmentation later and the other which doesnot do data augmentation. 
## The latter will be used for both training and validation to create a baseline model

In [ ]:
def display_aug(generator, img_path, target_size = (150,150), nb_img=4, save = None):
    assert nb_img < 7, 'please keep nb_img under 7'
    assert len(target_size)==2, 'Target size must be of (height, length)'

    img = load_img(img_path, grayscale=False, target_size=target_size)
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    
    fig, axes = plt.subplots(1, nb_img, figsize=(int(3*nb_img),int(5*nb_img)), sharex=False)

    axes = axes.flatten()

    idx = 0
    for batch in generator.flow(x, batch_size=1):
        axes[idx].imshow(array_to_img(batch[0]))
        axes[idx].axis('off')
        idx += 1
        if idx > nb_img-1:
            break
    if save : plt.savefig(save)
    plt.show()

In [ ]:
batch_size = 1

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip = True,
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
idx = 0
print('Using the augmentation')
for specie in baseline_species[:-3]:
    display_aug(train_datagen, baseline_dir+'train/'+specie+'/'+os.listdir(baseline_dir+'train/'+specie)[idx], save = png_dir+'image_aug_keras_1.png')
    
print('Using the no augmentation ')
for specie in baseline_species:
    display_aug(test_datagen, baseline_dir+'train/'+specie+'/'+os.listdir(baseline_dir+'train/'+specie)[idx])
    

## Lets create train and validation directories with CLAHE images

In [ ]:
def build_clahe_dir(input_dir, output_dir):
  assert os.path.exists(input_dir), 'input_dir doesnot exist'
  
  if output_dir[-1]!='/': output_dir+'/'
  if input_dir[-1]!='/': input_dir+'/'

  species = os.listdir(input_dir) 

  assert os.path.isdir(input_dir+species[0]), 'input_dir must contain directories of images'
  assert (not os.path.exists(output_dir)) , 'output_dir already exists, consider removing it first'

  os.mkdir(output_dir)
  print('Looking for images and returning their CLAHE image')

  i = 1
  for specie in species :
    os.mkdir(output_dir+specie)
    print('Dealing with ',i,'th specie:', specie.split('-')[1], '\nFound {} images'.format(len(os.listdir(input_dir+specie))))
    i += 1
    for file in os.listdir(input_dir+specie) :

      img = cv2.imread(input_dir+specie+'/'+file, 1)
      new = CLAHE_rgb(img)
      name = output_dir+specie+'/clahe_'+str(file).split('.')[0]+'.jpg'
      cv2.imwrite(name, new)


build_clahe_dir(input_dir+'sep_images/train/',input_dir+'sep_images/train_clahe/')
build_clahe_dir(input_dir+'sep_images/validation/',input_dir+'sep_images/validation_clahe/')